# Estimate Customer Lifetime Value using Markov Chain

**Outline**

* [Introduction](#intro)
* [Read Data](#read)
* [Reformat Data](#format)
* [Some EDA](#eda)
* [CLV model building using Recency Frequency Migration Model](#model)
    * [Get recency and frequency table with transition probability](#rf)
    * [Get Transition Matrix](#transition_max)
    * [Get Customer Lifetime Value](#clv)
* [Reference](#refer)    

---

According to previous [post about Customer Lifetime Value](http://nbviewer.jupyter.org/github/johnnychiuchiu/Machine-Learning/blob/master/CustomerLifetimeValue/CustomerLifetimeValue.ipynb), we know that migration model, more specifically, markov chain, can help us estimate the CLV based on the probabillity of the customers moving to each of the state. Let's figure how do we actually model CLV using the [ecommerce data I found on Kaggle](https://www.kaggle.com/lissetteg/ecommerce-dataset/data).

## <a id='read'>Read Data</a>

In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import inv

In [94]:
df_raw = pd.read_csv('_data/data-2.csv')
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In the table above, we can see that each row represent an item in a order from some particular customer. To calculate the customer lifetime value, we would like the table to be in the order level instead of product level. That's what we are going to do in the next step.

## <a id='format'>Reformat Data</a>

In [100]:
def data_manipulate(df):
    """
    change data shape and type
    
    Returns:
        pd.DataFrame: dataframe with a transaction per row
    """
    
    df['amount'] = df['Quantity']*df['UnitPrice']
    df2 = df.groupby(['CustomerID','InvoiceNo','InvoiceDate']).agg({'amount':np.sum}).reset_index().query("amount>0").reset_index(drop=True)
    df2['InvoiceDate'] = pd.to_datetime(df2['InvoiceDate'])    
    df2['date'] = df2['InvoiceDate'].apply(lambda x: x.date())  
    df2['CustomerID'] = df2['CustomerID'].apply(lambda x: str(int(x))) #.astype(int).astype(str)
    df2['month'] = df2['InvoiceDate'].apply(lambda x: x.month)
    df2 = df2.sort_values(by=['CustomerID','InvoiceDate'])
    
    return df2        

In [101]:
df = data_manipulate(df_raw)

In [102]:
df.head()

,CustomerID,InvoiceNo,InvoiceDate,amount,date,month
0,12346,541431,2011-01-18 10:01:00,77183.60,2011-01-18,1
1,12347,537626,2010-12-07 14:57:00,711.79,2010-12-07,12
2,12347,542237,2011-01-26 14:30:00,475.39,2011-01-26,1
3,12347,549222,2011-04-07 10:43:00,636.25,2011-04-07,4
4,12347,556201,2011-06-09 13:01:00,382.52,2011-06-09,6


In the table, we can see that each row represent an order from a particular customer. We also have the order date as well as amount per order, which are essential information when calculating customer lifetime value.

## <a id='eda'>Some EDA</a>

> **What is the time range of the data**

In [7]:
df['InvoiceDate'].describe()

count                   18562
unique                  17282
top       2011-10-21 14:41:00
freq                        4
first     2010-12-01 08:26:00
last      2011-12-09 12:50:00
Name: InvoiceDate, dtype: object

> **How many customers in the dataset**

In [8]:
len(df['CustomerID'].unique())

4338

> **How many unique customers are there in each month**

In [9]:
df.groupby('month').agg({'CustomerID':'nunique'}).reset_index()

,month,CustomerID
0,1,741
1,2,758
2,3,974
3,4,856
4,5,1056
5,6,991
6,7,949
7,8,935
8,9,1266
9,10,1364


> **How many order do each customer make on average?**
> **How much do people usually spend on each order on average?**

In [97]:
df.groupby('CustomerID').agg({'InvoiceNo':'nunique', 'amount': np.mean}).reset_index().describe()

,InvoiceNo,amount
count,4338.000000,4338.000000
mean,4.272015,418.710140
std,7.697998,1796.481888
min,1.000000,3.450000
25%,1.000000,178.450500
50%,2.000000,292.552000
75%,5.000000,428.890000
max,209.000000,84236.250000


> **What is the return purchase period?**

In [152]:
def get_purchase_period(df):
    """get the number of days between each order for people who have made multiple purchases"""
    
    # keep rows from cid with multiple transaction records
    df_multi_id = df.groupby('CustomerID').agg({'InvoiceNo':'nunique'}).reset_index().query('InvoiceNo>1')['CustomerID']    
    df_multi = df[df['CustomerID'].isin(list(df_multi_id))].copy()
    
    # get the number of days between each order
    df_multi['date_prev'] = df_multi['date'].shift(1)
    df_multi['date_diff'] = df_multi['date'] - df_multi['date_prev']
    df_multi['date_diff'] = df_multi['date_diff'].apply(lambda x: x.days)
    
    # get rid of first row per customerId
    df_multi['cid_shift'] = df_multi['CustomerID'].shift(1)
    df_multi['is_first_row'] = df_multi['CustomerID']!=df_multi['cid_shift']
    df_multi = df_multi[df_multi['is_first_row']==False]
    
    return df_multi

In [153]:
df_return = get_purchase_period(df)

In [156]:
df_avg_return= df_return.groupby('CustomerID').agg({'date_diff':np.mean}).reset_index()

In [159]:
df_avg_return['date_diff'].describe()

count    2845.000000
mean       72.523273
std        65.463660
min         0.000000
25%        29.400000
50%        53.400000
75%        91.750000
max       366.000000
Name: date_diff, dtype: float64

We can see that the average return purchase period is 73 days. It means that for people who have made multiple purchases, the average number of days between each order is **73 days**.

The median number of days between each order is **53 days**.

## <a id='model'>CLV model building using Recency Frequency Migration Model</a>

Unlike the previous post, instead of using buy or not-buy as customer state, we can also use recency and frequency to separate all the customer's into different states. To do this, the first step is to define a snapshot of time that can separate our transaction record into previous and current period. The reason why we do this is that we want to use the 
return purchase ratio to estimate the transition probability that we use in the transition matrix. 

Intuitively, assuming that this probability hold true for any period afterwards, we can use it to estimate the customer lifetime value. There are several assumption and fix parameters we need to make to calculate the final value.

To estimate CLV using Recency Frequency migration model, we need the following information

**Assumption**
* **Define snapshot of time**: before/after 2010 beginning of Oct
* **Define the length of period in Recency**: Let's define the recency length to be a month. Recency<1 to be some one who made a purchase in 2010 Sep. 
* **Define different groups of Recency and Frequency**: 
    * R < 1, F=1    
    * R < 1, F>1    
    * $1\le R<2$, F=1
    * $1\le R<2$, F$\ge$1    
    * $2\le R<3$, F$\ge$1        
    * $3\le R<4$, F$\ge$1    
    * $4\le R$, F$\ge$1    
* **Define Discount Rate**: let's assume the discount rate = 10%

**Input of the model**
* **Initial Vector**: Get the number of customers in each state at time 0
* **Transition Matrix**: Calculate the transition matrix according to the assumption we made
* **Value Vector**: The marketing costs at time t are c and the m is the net contribution, so that a customer who does not buy at t generates −c profit while one who does buy generates m − c.
    * **marketing cost**: the cost of acquiring this customer at each of the time.
    * **net contribution**: use the median total amount for all the customers, assuming that if they remain to be our customer, they would spent this amount in each period.

**Output of the model**

* **CLV vector**: each element indicate the CLV for the customers in each state.

---

In [13]:
def get_recency(df, snapshot_date, col_cid, col_date):
    """
    Get the recency of customers according to last purchase date.
    
    Args:
        df (pd.DataFrame): the transaction dataframe
        snapshot_date (datetime.date): 
            the snapshot date we want to use to estimate the transition probability
            we may want to use the snapshot date as today() minus some certain date to be our snapshot date in the future
        col_cid (str): the column name for customer ID. In this case, it's `CustomerID`    
        col_date (str): the column name for transaction date. In this case, it's `InvoiceDate`
        
    Returns:
        pd.DataFrame: a dataframe with column [CustomerID, Recency]
        
        R=0 indicates that these customers are our current buyers
        R>1, ...etc indicates that these are customers that didn't make any purchase in our current period            
    """

    # get the most recent transaction for each cid
    df_recent = df.groupby(col_cid).agg({'date':'max'}).reset_index()

    # get recency column
    df_recent['recency'] = df_recent['date'].apply(lambda x: label_recency(x, snapshot_date))
    
    return df_recent[['CustomerID', 'recency']]
    
    
def label_recency(date, snapshot_date):
    """
    match date to recency measure
    """
    
    if (date >= snapshot_date):
        recency = 'R=0' 
    elif (date >= (snapshot_date- pd.DateOffset(months=1)).date()):
        recency = 'R<1' 
    elif (date >= (snapshot_date- pd.DateOffset(months=2)).date()):
        recency = '1<=R<2'
    elif (date >= (snapshot_date- pd.DateOffset(months=3)).date()):        
        recency = '2<=R<3'  
    elif (date >= (snapshot_date- pd.DateOffset(months=4)).date()):        
        recency = '3<=R<4'         
    else:
        recency = 'R>=4'         

    return recency 
 

In [14]:
def get_frequency(df, col_cid, col_invoice):
    """
    Get the frequency of customers according to total number of purchases
    
    Args:
        df (pd.DataFrame): the transaction dataframe
        col_cid (str): the column name for customer ID. In this case, it's `CustomerID`            
        col_invoice (str): the column name for transaction ID. In this case, it's `InvoiceNo`

    Returns:
        pd.DataFrame: a dataframe with column [CustomerID, Frequenct]
    
    """

    df_freq = df.groupby(col_cid).agg({col_invoice:'nunique'}).reset_index().rename(columns={'InvoiceNo':'TranCount'})
    df_freq['frequency'] = ['F=1' if t == 1 else 'F>1' for t in df_freq['TranCount']]
    
    return df_freq

In [64]:
def merge_date(df_freq, df_recency, col_cid = 'CustomerID'):
    """merge all the clv related table together and keep one record for each customer"""
    
    customer_df = pd.merge(df_freq, df_recency, on=col_cid, how='outer')
    
    # get state label
    customer_df['state'] = customer_df.apply(lambda row: label_state(row.recency, row.frequency), axis=1)
    
    return customer_df

In [17]:
def label_state(r, f):
    """label state of customer by recency and frequency"""
        
    if r=='R<1' and f=='F=1':
        state = "state 1: R<1 F=1"
    elif r=='R<1' and f=='F>1':        
        state = "state 2: R<1 F>1"    
    elif r=='1<=R<2' and f=='F=1':        
        state = "state 3: 1<=R<2 F=1"    
    elif r=='1<=R<2' and f=='F>1':        
        state = "state 4: 1<=R<2 F>1"    
    elif r=='2<=R<3':        
        state = "state 5: 2<=R<3 F>=1"    
    elif r=='3<=R<4':        
        state = "state 6: 3<=R<4 F>=1" 
    elif r=='R>=4':        
        state = "state 7: R>=4 F>=1"      

    return state

In [43]:
def count_buyer(df_initial, df_current, col_cid):
    """For each state, get the number of people who purchase in current period"""
    
    df_buyer = pd.DataFrame()
    
    for s in list(df_initial['state'].unique()):
        
        df_initial_filter = df_initial[df_initial['state']==s]
        df_current_filter = df_current[df_current[col_cid].isin(df_initial_filter[col_cid])].copy()
        
        # get unique count
        unique_count = len(df_current_filter[col_cid].unique())
        
        # get total dollar
        total_dollar = sum(df_current_filter['amount'])
        
        df_buyer_state = pd.DataFrame({'state': [s],
                                      'BuyerCount': [unique_count],
                                      'TotalDollars': [total_dollar]})
        
        df_buyer = pd.concat([df_buyer, df_buyer_state])
        
    return df_buyer.reset_index(drop=True)

In [61]:
def get_rf(df_previous, snapshot_date, col_cid='CustomerID', col_invoice='InvoiceNo', col_date = 'date'):
    """get recency and frequency group for the previous period dataframe"""
    
    # Generate frequency dataframe
    df_freq = get_frequency(df_previous, 
                        col_cid, 
                        col_invoice)
    
    # Generate recency dataframe
    df_recency = get_recency(df_previous, 
                         snapshot_date, 
                         col_cid, 
                         col_date)

    # Merging table together and generate state label
    df_initial = merge_date(df_freq, df_recency, col_cid)
       
    return df_initial

In [60]:
def get_transition(df, snapshot_date, col_cid='CustomerID', col_invoice='InvoiceNo', col_date = 'date'):
    """get the dataframe with transition probability"""
    
    # Separate data into previous and current period
    df_current  = df[df['date']>= snapshot_date]
    df_previous = df[df['date']< snapshot_date]
    
    # get recency and frequency group for the previous period dataframe
    df_initial = get_rf(df_previous, 
                        snapshot_date, 
                        col_cid,
                        col_invoice, 
                        col_date)
    
    # get initial vector
    df_customer = df_initial.groupby('state').agg({col_cid:'nunique'}).reset_index().rename(columns = {col_cid: 'CustomerCount'})
       
    # For each state, get the number of people who purchase in current period
    df_buyer = count_buyer(df_initial, df_current, col_cid)
        
    # get final dataframe
    df_final = pd.merge(df_customer, df_buyer, on='state')
    df_final['BuyRate'] = df_final['BuyerCount']/df_final['CustomerCount']
    df_final['AvgDollars'] = df_final['TotalDollars']/df_final['CustomerCount']  
    
    return df_final

## <a id='rf'>Get recency and frequency table with transition probability</a>

* **Get Initial Vector**: CustomerCount is our initial vector
* **Get Transition Probability**: BuyCount to the CustomerCount is our Transition Probability for each state
* **Get Value Vector**: Using the average total dollar spent per customer as the net contribution for customer in each state

In [70]:
snapshot_date = pd.to_datetime('2011/11/01 00:00:00').date()
df_final = get_transition(df, snapshot_date, col_cid='CustomerID', col_invoice='InvoiceNo', col_date = 'date')

In [71]:
df_final

,state,CustomerCount,BuyerCount,TotalDollars,BuyRate,AvgDollars
0,state 1: R<1 F=1,310,76,37646.22,0.245161,121.439419
1,state 2: R<1 F>1,1054,636,836151.70,0.603416,793.312808
2,state 3: 1<=R<2 F=1,210,54,20815.98,0.257143,99.123714
3,state 4: 1<=R<2 F>1,567,310,215603.56,0.546737,380.253192
4,state 5: 2<=R<3 F>=1,339,124,58426.18,0.365782,172.348614
5,state 6: 3<=R<4 F>=1,263,85,58519.57,0.323194,222.507871
6,state 7: R>=4 F>=1,1231,259,273910.77,0.210398,222.510780


In [72]:
initial_vector = np.array(df_final['CustomerCount'])
initial_vector

array([ 310, 1054,  210,  567,  339,  263, 1231])

In [73]:
value_vector = np.array(df_final['AvgDollars'])
value_vector

array([ 121.43941935,  793.31280835,   99.12371429,  380.25319224,
        172.34861357,  222.50787072,  222.51077985])

In [75]:
transition_prob = np.array(df_final['BuyRate'])
transition_prob

array([ 0.24516129,  0.60341556,  0.25714286,  0.54673721,  0.36578171,
        0.32319392,  0.21039805])

## <a id='transition_max'> Get Transition Matrix</a>

In [76]:
tran_mat = np.matrix(
    ((0, transition_prob[0], 1-transition_prob[0], 0, 0, 0, 0),
     (0, transition_prob[1], 0, 1-transition_prob[1], 0, 0, 0),
     (0, transition_prob[2], 0, 0, 1-transition_prob[2], 0, 0),
     (0, transition_prob[3], 0, 0, 1-transition_prob[3], 0, 0),
     (0, transition_prob[4], 0, 0, 0, 1-transition_prob[4], 0),
     (0, transition_prob[5], 0, 0, 0, 0, 1-transition_prob[5]),
     (0, transition_prob[6], 0, 0, 0, 0, 1-transition_prob[6]))
)

tran_mat

matrix([[ 0.        ,  0.24516129,  0.75483871,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.60341556,  0.        ,  0.39658444,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.25714286,  0.        ,  0.        ,  0.74285714,
          0.        ,  0.        ],
        [ 0.        ,  0.54673721,  0.        ,  0.        ,  0.45326279,
          0.        ,  0.        ],
        [ 0.        ,  0.36578171,  0.        ,  0.        ,  0.        ,
          0.63421829,  0.        ],
        [ 0.        ,  0.32319392,  0.        ,  0.        ,  0.        ,
          0.        ,  0.67680608],
        [ 0.        ,  0.21039805,  0.        ,  0.        ,  0.        ,
          0.        ,  0.78960195]])

## <a id='clv'>Get Customer Lifetime Value</a>

Recall that the migration model formula for CLV is:

$$CLV = \sum_{t=0}^{\infty} \frac{P^tv}{(1+d)^t} = \Big[ \sum_{t=0}^{\infty} \Big( \frac{P}{1+d} \Big)^t \Big] v = \Big( I - \frac{P}{1+d} \Big)^{-1} v$$

In [85]:
def get_clv(df_final, tran_mat, value_vector, d):
    """calculate clv using migration model formula and combine with the state calculate using function get_transition"""
    
    clv = np.dot(inv(np.identity(7) - tran_mat/(1+d)), value_vector)
    df_clv = pd.DataFrame(np.transpose(clv))
    df_clv['state'] = df_final['state']
    df_clv = df_clv.rename(columns = {0: 'CLV'})[['state', 'CLV']]
    
    return df_clv

In [88]:
df_clv1 = get_clv(df_final, tran_mat, value_vector, d=0.1)
df_clv1

,state,CLV
0,state 1: R<1 F=1,5023.981862
1,state 2: R<1 F>1,6277.199480
2,state 3: 1<=R<2 F=1,5105.554754
3,state 4: 1<=R<2 F>1,5659.615641
4,state 5: 2<=R<3 F>=1,5240.492297
5,state 6: 3<=R<4 F>=1,5169.944391
6,state 7: R>=4 F>=1,5043.434997


In [89]:
df_clv2 = get_clv(df_final, tran_mat, value_vector, d=0.05)
df_clv2

,state,CLV
0,state 1: R<1 F=1,10257.556451
1,state 2: R<1 F>1,11567.142672
2,state 3: 1<=R<2 F=1,10342.748934
3,state 4: 1<=R<2 F>1,10925.107107
4,state 5: 2<=R<3 F>=1,10474.959338
5,state 6: 3<=R<4 F>=1,10385.528354
6,state 7: R>=4 F>=1,10243.320108


From the result, we can see that it actually make sense in terms of relative scale. For example, the customer lifetime value for people who are in state 2 is higher than people who are in state 1; the customer lifetime value for people who are in state 4 is higher than people who are in state 3 since their average contribution and return purchase rate are higher.

Also notice that by using different discount rate, the absolute scale of the CLV result is actually a lot different. This will cause some issue if we want to use the absolute value to determine how much we want to spend on people who are in different state.

## <a id='refer'>Reference</a>

* [Data Source](https://www.kaggle.com/lissetteg/ecommerce-dataset)